#처음부터 다시 시작해야 할 경우, 한번에 라이브러리 다운할 수 있도록 만듦.

pip install -r requirements.txt

### 01. 패키지 불러오기

In [3]:
from etl_module.connectors.weather_api import WeatherApiClient
from etl_module.connectors.mysql import MySqlClient


import pandas as pd 
import os
from dotenv import load_dotenv
load_dotenv()

True

#### .env 환경변수 설정

In [10]:
# 환경변수 설정 
API_KEY = os.environ.get("API_KEY")
DB_SERVER_HOST = os.environ.get('DB_SERVER_HOST')  # 데이터베이스 서버의 호스트 이름 (로컬호스트로 설정)
DB_USERNAME = os.environ.get('DB_USERNAME')      # 데이터베이스 아이디 
DB_PASSWORD = os.environ.get('DB_PASSWORD')      # 데이터베이스 비밀번호
DB_DATABASE = os.environ.get('DB_DATABASE') # 사용할 데이터베이스 이름
DB_PORT = os.environ.get('DB_PORT')              # 데이터베이스 연결을 위한 포트 (Default: 3306)


In [11]:
DB_DATABASE

'test_db'

### 02. Weather, MySQL 클래스를 객체(인스턴스)화

In [12]:
# Client 
weather_api_client = WeatherApiClient(api_key = API_KEY) 
my_sql_client = MySqlClient(
    server_name=DB_SERVER_HOST, 
    database_name=DB_DATABASE, 
    username=DB_USERNAME, 
    password=DB_PASSWORD, 
    port=DB_PORT
) 

In [16]:
weather_api_client.base_url

'http://api.openweathermap.org/data/2.5'

In [17]:
# 테스트
weather_api_client.get_city(city_name='seoul')

{'coord': {'lon': 126.9778, 'lat': 37.5683},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 9.51,
  'feels_like': 8.57,
  'temp_min': 8.66,
  'temp_max': 9.76,
  'pressure': 1023,
  'humidity': 53,
  'sea_level': 1023,
  'grnd_level': 1016},
 'visibility': 10000,
 'wind': {'speed': 2.06, 'deg': 160},
 'clouds': {'all': 75},
 'dt': 1732080002,
 'sys': {'type': 1,
  'id': 8105,
  'country': 'KR',
  'sunrise': 1732054633,
  'sunset': 1732090711},
 'timezone': 32400,
 'id': 1835848,
 'name': 'Seoul',
 'cod': 200}

### 03. 추출(Extract) 

In [18]:
# 서울말고도 여러 도시에 대해 날씨 정보를 추출해볼까요? 
def extract_weather(weather_api_client: WeatherApiClient) -> pd.DataFrame:
    """
    여러 도시의 날씨 데이터를 추출합니다.

    Parameters:
    - weather_api_client (WeatherApiClient): API에서 날씨 데이터를 가져오기 위한 클라이언트.

    Returns:
    - pd.DataFrame: 지정된 도시들의 날씨 데이터를 포함하는 DataFrame.
    """
    cities = ["seoul", "busan", "sejong", "daegu", "incheon", "daejeon", "ulsan"]
    weather_data = []
    for city_name in cities:
        data  = weather_api_client.get_city(city_name=city_name)
        weather_data.append(data)
    df = pd.json_normalize(weather_data)
    return df

In [19]:
df = extract_weather(weather_api_client=weather_api_client)
df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,wind.gust
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1732080002,32400,1835848,Seoul,200,126.9778,37.5683,...,1016,2.06,160,75,1.0,8105.0,KR,1732054633,1732090711,NaN
1,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1732080657,32400,1838524,Busan,200,129.0403,35.1028,...,1020,4.63,300,0,1.0,8086.0,KR,1732053809,1732090545,NaN
2,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1732080407,32400,1842616,Sejong,200,127.2871,36.4817,...,1022,1.93,291,92,1.0,8131.0,KR,1732054411,1732090784,3.71
3,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1732080454,32400,1835327,Daegu,200,128.5500,35.8000,...,1008,4.63,300,0,1.0,8124.0,KR,1732054018,1732090572,NaN
4,"[{'id': 500, 'main': 'Rain', 'description': 'l...",stations,10000,1732080499,32400,1843561,Incheon,200,126.4161,37.4500,...,1021,3.09,150,100,1.0,8093.0,KR,1732054752,1732090862,NaN


### 04. 변환(Transform)

In [31]:
# 전처리 코드를 함수로 만들어볼게요. 
def transform_weather(df: pd.DataFrame) -> pd.DataFrame:
    """
    날씨 데이터를 변환하고 전처리합니다.

    Parameters:
    - df (pd.DataFrame): 원본 날씨 데이터를 포함하는 DataFrame.
    Returns:
    - pd.DataFrame: 선택된 컬럼과 이름이 변경된 데이터로 구성된 변환된 DataFrame.
    """
    df["measured_at"] = pd.to_datetime(df["dt"], unit="s") + pd.Timedelta(hours=9)  # 한국시간
    df["dt"] = df["measured_at"].dt.strftime("%Y%m%d")  # 기준년월일 (YYYYMMDD)
    df["time"] = df["measured_at"].dt.strftime("%H%M%S")  # 기준년월일 (HHHHMMSS)
    df_selected = df[["dt", "time", "measured_at", "id", "name", "main.temp", "main.humidity", "wind.speed"]]
    df_selected = df_selected.rename(columns={
        "name": "city", 
        "main.temp": "temperature", 
        "main.humidity": "humidity", 
        "wind.speed": "wind_speed"
    })
    return df_selected

In [32]:
clean_df = transform_weather(df)
clean_df.head()

/var/folders/29/5wmqbm4j3x10qv5vgm_bm8f80000gn/T/ipykernel_26266/4282564830.py:12: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df["measured_at"] = pd.to_datetime(df["dt"], unit="s") + pd.Timedelta(hours=9)  # 한국시간


,dt,time,measured_at,id,city,temperature,humidity,wind_speed
0,19700823,153200,1970-08-23 15:32:00,1835848,Seoul,9.74,53,2.06
1,19700823,153200,1970-08-23 15:32:00,1838524,Busan,13.99,41,4.63
2,19700823,153200,1970-08-23 15:32:00,1842616,Sejong,9.24,51,1.93
3,19700823,153200,1970-08-23 15:32:00,1835327,Daegu,13.47,35,4.63
4,19700823,153200,1970-08-23 15:32:00,1843561,Incheon,8.93,71,3.09


### 05. 적재(Load)

In [33]:
from sqlalchemy import MetaData, Table, Column, String, DateTime, Integer, Float

In [37]:
def load_weather(df: pd.DataFrame, my_sql_client: MySqlClient, method: str = "upsert") -> None:
    """
    변환된 날씨 데이터를 MySQL 데이터베이스에 로드합니다.

    Parameters:
    - df (pd.DataFrame): 변환된 날씨 데이터를 포함하는 DataFrame.
    - my_sql_client (MySqlClient): MySQL 데이터베이스와 상호작용하는 클라이언트.
    - table (Table): 데이터를 로드할 대상 데이터베이스 테이블.
    - metadata (MetaData): 테이블 정의에 대한 SQLAlchemy 메타데이터 객체.
    - method (str, optional): 데이터 삽입 방법을 지정합니다.
                            옵션: "insert", "upsert", "overwrite".
                            기본값은 "upsert"입니다.
    """
    metadata = MetaData()
    table = Table(
        "daily_weather",
        metadata,
        Column("dt", String(8), nullable=False, primary_key=True),
        Column("time", String(6), nullable=False, primary_key=True),
        Column("measured_at", DateTime, nullable=False),
        Column("id", Integer, primary_key=True),
        Column("city", String(100), nullable=True),
        Column("temperature", Float, nullable=True),
        Column("humidity", Integer, nullable=True),
        Column("wind_speed", Float, nullable=True),
    )
    if method == "insert":
        my_sql_client.insert(df=df, table=table, metadata=metadata)
    elif method == "upsert":
        my_sql_client.upsert(df=df, table=table, metadata=metadata)
    elif method == "overwrite":
        my_sql_client.overwrite(df=df, table=table, metadata=metadata)
    else:
        raise Exception("올바른 method를 설정해주세요: [insert, upsert, overwrite]")


In [38]:
load_weather(df=clean_df, my_sql_client=my_sql_client)

### 06. ETL (추출(Extract), 변환(Transform), 적재(Load)) 메인 스크립트 실행

In [39]:
df = extract_weather(weather_api_client=weather_api_client)
clean_df = transform_weather(df)
load_weather(df=clean_df, my_sql_client=my_sql_client, method="overwrite")

In [40]:
import os
from dotenv import load_dotenv
from etl_module.connectors.weather_api import WeatherApiClient
from etl_module.connectors.mysql import MySqlClient
from etl_module.assets.weather import extract_weather, transform_weather, load_weather

def main():
    load_dotenv()
    API_KEY = os.environ.get("API_KEY")
    DB_SERVER_HOST = os.environ.get('DB_SERVER_HOST') 
    DB_USERNAME = os.environ.get('DB_USERNAME')
    DB_PASSWORD = os.environ.get('DB_PASSWORD') 
    DB_DATABASE = os.environ.get('DB_DATABASE')
    DB_PORT = os.environ.get('DB_PORT')

    weather_api_client = WeatherApiClient(api_key = API_KEY) 
    my_sql_client = MySqlClient(
        server_name=DB_SERVER_HOST, 
        database_name=DB_DATABASE, 
        username=DB_USERNAME, 
        password=DB_PASSWORD, 
        port=DB_PORT
    ) 

    # ETL 실행
    df = extract_weather(weather_api_client=weather_api_client)
    clean_df = transform_weather(df)
    load_weather(df=clean_df, my_sql_client=my_sql_client)


In [41]:
main()